# Reranking Methods in RAG Systems

## Overview
Reranking is a crucial step in Retrieval-Augmented Generation (RAG) systems that aims to improve the relevance and quality of retrieved documents. It involves reassessing and reordering initially retrieved documents to ensure that the most pertinent information is prioritized for subsequent processing or presentation.

## Motivation
The primary motivation for reranking in RAG systems is to overcome limitations of initial retrieval methods, which often rely on simpler similarity metrics. Reranking allows for more sophisticated relevance assessment, taking into account nuanced relationships between queries and documents that might be missed by traditional retrieval techniques. This process aims to enhance the overall performance of RAG systems by ensuring that the most relevant information is used in the generation phase.

## Key Components
Reranking systems typically include the following components:

1. Initial Retriever: Often a vector store using embedding-based similarity search.
2. Reranking Model: This can be either:
   - A Large Language Model (LLM) for scoring relevance
   - A Cross-Encoder model specifically trained for relevance assessment
3. Scoring Mechanism: A method to assign relevance scores to documents
4. Sorting and Selection Logic: To reorder documents based on new scores

## Method Details
The reranking process generally follows these steps:

1. Initial Retrieval: Fetch an initial set of potentially relevant documents.
2. Pair Creation: Form query-document pairs for each retrieved document.
3. Scoring:
   - LLM Method: Use prompts to ask the LLM to rate document relevance.
   - Cross-Encoder Method: Feed query-document pairs directly into the model.
4. Score Interpretation: Parse and normalize the relevance scores.
5. Reordering: Sort documents based on their new relevance scores.
6. Selection: Choose the top K documents from the reordered list.

## Benefits of this Approach
Reranking offers several advantages:

1. Improved Relevance: By using more sophisticated models, reranking can capture subtle relevance factors.
2. Flexibility: Different reranking methods can be applied based on specific needs and resources.
3. Enhanced Context Quality: Providing more relevant documents to the RAG system improves the quality of generated responses.
4. Reduced Noise: Reranking helps filter out less relevant information, focusing on the most pertinent content.

## Conclusion
Reranking is a powerful technique in RAG systems that significantly enhances the quality of retrieved information. Whether using LLM-based scoring or specialized Cross-Encoder models, reranking allows for more nuanced and accurate assessment of document relevance. This improved relevance translates directly to better performance in downstream tasks, making reranking an essential component in advanced RAG implementations.

The choice between LLM-based and Cross-Encoder reranking methods depends on factors such as required accuracy, available computational resources, and specific application needs. Both approaches offer substantial improvements over basic retrieval methods and contribute to the overall effectiveness of RAG systems.

<div style="text-align: center;">

<img src="../images/reranking-visualization.svg" alt="rerank llm" style="width:100%; height:auto;">
</div>

<div style="text-align: center;">

<img src="../images/reranking_comparison.svg" alt="rerank llm" style="width:100%; height:auto;">
</div>

# Package Installation and Imports

The cell below installs all necessary packages required to run this notebook.


In [ ]:
# Install required packages
#!pip install langchain langchain-openai python-dotenv sentence-transformers

In [ ]:
# !pip install langchain langchain-openai langchain-community langchain-core langchain-experimental\
#             python-dotenv sentence-transformers pydantic faiss-cpu \
#             langchain-huggingface sentence-transformers langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.1/210.1 kB 4.8 MB/s eta 0:00:00


In [ ]:
# Install required packages
!pip install langchain langchain-community langchain-openai \
    langchain-text-splitters langchain-core faiss-cpu \
    pymupdf rank_bm25 deepeval pydantic langchain-classic \
    langchain-experimental langchain-openai python-dotenv langchain-huggingface sentence-transformers langchain-groq

  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langsmith-0.0.92-py3-none-any.whl.metadata (9.9 kB)
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versions of langchain-text-splitters to determine which version is compatible with other requirements. This could take a while.
  Using cached langchain_text_splitters-0.3.11-py3-none-any.whl.metadata (1.8 kB)
  Using cached langchain_text_splitters-0.3.10-py3-none-any.whl.

In [ ]:
# Clone the repository to access helper functions and evaluation modules
!git clone https://github.com/NirDiamant/RAG_TECHNIQUES.git
import sys
sys.path.append('RAG_TECHNIQUES')
# If you need to run with the latest data
# !cp -r RAG_TECHNIQUES/data .

fatal: destination path 'RAG_TECHNIQUES' already exists and is not an empty directory.


In [ ]:
%pwd

'/content'

In [ ]:
import os
import sys
from dotenv import load_dotenv
from typing import List, Dict, Any, Tuple
from langchain_core.documents import Document
from langchain_core.retrievers import BaseRetriever
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

from langchain_classic.chains import create_retrieval_chain
from langchain_classic.retrievers import ContextualCompressionRetriever
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from pydantic import BaseModel, Field
from sentence_transformers import CrossEncoder

# current_dir = os.getcwd()

# # Append the current directory to sys.path so Python can find your files
# sys.path.append(current_dir)

# Load helper functions
from helper_functions import *
from evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the Groq API key environment variable
#os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')

### Define the document's path

In [ ]:
# Download required data files
import os
os.makedirs('data', exist_ok=True)

# Download the PDF document used in this notebook
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
!wget -O data/Understanding_Climate_Change.pdf https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf


--2026-01-24 14:30:17--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 206372 (202K) [application/octet-stream]
Saving to: ‘data/Understanding_Climate_Change.pdf’

data/Understanding_ 100%[===================>] 201.54K  --.-KB/s    in 0.06s   

2026-01-24 14:30:18 (3.34 MB/s) - ‘data/Understanding_Climate_Change.pdf’ saved [206372/206372]

--2026-01-24 14:30:18--  https://raw.githubusercontent.com/NirDiamant/RAG_TECHNIQUES/main/data/Understanding_Climate_Change.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185

In [ ]:
path = "data/Understanding_Climate_Change.pdf"

### Create a vector store

In [ ]:
!pip install pypdf
vectorstore = encode_pdf(path)

/content/helper_functions.py:90: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


## Method 1: LLM based function to rerank the retrieved documents

<div style="text-align: center;">

<img src="../images/rerank_llm.svg" alt="rerank llm" style="width:40%; height:auto;">
</div>

### Create a custom reranking function


In [ ]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
from typing import List
from langchain_core.documents import Document
from pydantic import BaseModel, Field
from langchain_core.prompts import PromptTemplate

# Load environment variables from a .env file first
load_dotenv()

# Get a free API key from https://console.groq.com/
os.environ["GROQ_API_KEY"] = os.getenv('GROQ_API_KEY')

class RatingScore(BaseModel):
    relevance_score: float = Field(..., description="The relevance score of a document to a query.")

def rerank_documents(query: str, docs: List[Document], top_n: int = 3) -> List[Document]:
    prompt_template = PromptTemplate(
        input_variables=["query", "doc"],
        template="""On a scale of 1-10, rate the relevance of the following document to the query. Consider the specific context and intent of the query, not just keyword matches.
        Query: {query}
        Document: {doc}
        Relevance Score:"""
    )

    #llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)

    # Initialize Llama 3 as a free, production-grade evaluator
    llm = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile",  max_tokens=4000)
    llm_chain = prompt_template | llm.with_structured_output(RatingScore)

    scored_docs = []
    for doc in docs:
        input_data = {"query": query, "doc": doc.page_content}
        score = llm_chain.invoke(input_data).relevance_score
        try:
            score = float(score)
        except ValueError:
            score = 0  # Default score if parsing fails
        scored_docs.append((doc, score))

    reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in reranked_docs[:top_n]]

### Example usage of the reranking function with a sample query relevant to the document


In [ ]:
query = "What are the impacts of climate change on biodiversity?"
initial_docs = vectorstore.similarity_search(query, k=15)

print("Testing reranking...")
print(f"Retrieved {len(initial_docs)} initial documents")
reranked_docs = rerank_documents(query, initial_docs)

print(f"\nReranked to {len(reranked_docs)} documents")
# print first 3 initial documents
print("Top initial documents:")
for i, doc in enumerate(initial_docs[:3]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document


# Print results
print(f"Query: {query}\n")
print("Top reranked documents:")
for i, doc in enumerate(reranked_docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document

Top initial documents:

Document 1:
Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species 
distributions, and impacting ecosystem functions. Forests, grasslands, and deserts are 
experiencing shi...

Document 2:
protection, and habitat creation. 
Climate-Resilient Conservation 
Conservation strategies must account for climate change impacts to be effective. This 
includes identifying climate refugia, areas le...

Document 3:
The economic costs of climate change include damage to infrastructure, reduced agricultural 
productivity, health care costs, and lost labor productivity. Extreme weather events, such as 
hurricanes a...
Query: What are the impacts of climate change on biodiversity?

Top reranked documents:

Document 1:
Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species 
distributions, and impacting ecosystem functions. Forests, grasslands, and deserts are 
experiencing shi...

Document 2:
protecti

Testing

In [21]:
# Test if Groq LLM is working at all
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-versatile",
    max_tokens=8000
)

# Simple test
test_response = llm.invoke("What is climate change? Answer in 2 sentences.")
print("LLM Test Response:")
print(test_response.content)
print(f"\nResponse length: {len(test_response.content)} characters")

LLM Test Response:
Climate change refers to the long-term warming of the planet due to an increase in average global temperatures, primarily caused by human activities such as burning fossil fuels, deforestation, and other greenhouse gas emissions. This warming is leading to rising sea levels, more extreme weather events, and altered ecosystems, posing significant threats to the environment, human health, and the economy.

Response length: 406 characters


In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

# Create custom retriever
class CustomRetriever(BaseRetriever, BaseModel):
    vectorstore: Any = Field(description="Vector store for initial retrieval")

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(self, query: str, num_docs=3) -> List[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=15)
        return rerank_documents(query, initial_docs, top_n=num_docs)

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError("Async retrieval not implemented")


# Initialize
custom_retriever = CustomRetriever(vectorstore=vectorstore)

# Format docs function
def format_docs(docs):
    formatted = []
    for i, doc in enumerate(docs, 1):
        formatted.append(f"[Document {i}]\n{doc.page_content}\n")
    return "\n".join(formatted)

# Create a simple prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions based on the provided context. Be comprehensive and specific."),
    ("human", """Context:
{context}

Question: {question}

Please provide a detailed answer based on the context above.""")
])

# Create LLM
llm = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-versatile",
    max_tokens=8000
)

# Build the chain manually
def get_context(inputs):
    docs = custom_retriever._get_relevant_documents(inputs["question"])
    return format_docs(docs)

chain = (
    {
        "context": lambda x: get_context(x),
        "question": lambda x: x["question"]
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Test it
query = "What are the impacts of climate change on biodiversity?"
print(f"Question: {query}\n")
print("Generating answer...\n")

answer = chain.invoke({"question": query})

print("="*80)
print("ANSWER:")
print("="*80)
print(answer)
print("="*80)
print(f"\nAnswer length: {len(answer)} characters")

# Also show retrieved docs
print("\n" + "="*80)
print("RETRIEVED DOCUMENTS:")
print("="*80)
docs = custom_retriever._get_relevant_documents(query)
for i, doc in enumerate(docs, 1):
    print(f"\nDocument {i}:")
    print(doc.page_content[:400])
    print("...\n")

/tmp/ipython-input-3166084370.py:6: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class CustomRetriever(BaseRetriever, BaseModel):


Question: What are the impacts of climate change on biodiversity?

Generating answer...

ANSWER:
According to the provided context, climate change has significant impacts on biodiversity in both terrestrial and marine ecosystems. The effects of climate change on biodiversity can be summarized as follows:

**Terrestrial Ecosystems:**

1. **Shifts in habitat ranges**: Climate change is altering the ranges of habitats, leading to changes in the distribution of plant and animal species.
2. **Changes in species composition**: Forests, grasslands, and deserts are experiencing shifts in the composition of plant and animal species, which can lead to a loss of biodiversity.
3. **Disruption of ecological balance**: These changes can disrupt the ecological balance, leading to a decline in ecosystem functions and services.

**Marine Ecosystems:**

1. **Rising sea temperatures**: Increased sea temperatures are affecting marine biodiversity, from coral reefs to deep-sea habitats.
2. **Ocean acidific

In [24]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_groq import ChatGroq

# Custom Retriever (update to remove Pydantic warning)
from pydantic import ConfigDict

class CustomRetriever(BaseRetriever, BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)  # Updated for Pydantic v2

    vectorstore: Any = Field(description="Vector store for initial retrieval")

    def _get_relevant_documents(self, query: str, num_docs=3) -> List[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=15)
        return rerank_documents(query, initial_docs, top_n=num_docs)

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError("Async retrieval not implemented")


# Initialize retriever
custom_retriever = CustomRetriever(vectorstore=vectorstore)

# Format docs function
def format_docs(docs):
    formatted = []
    for i, doc in enumerate(docs, 1):
        formatted.append(f"[Document {i}]\n{doc.page_content}\n")
    return "\n".join(formatted)

# Create prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Answer questions based on the provided context. Be comprehensive and specific."),
    ("human", """Context:
{context}

Question: {question}

Please provide a detailed answer based on the context above.""")
])

# Create LLM
llm = ChatGroq(
    temperature=0,
    model_name="llama-3.3-70b-versatile",
    max_tokens=8000
)

# Build the chain
def get_context(inputs):
    docs = custom_retriever._get_relevant_documents(inputs["question"])
    return format_docs(docs)

qa_chain = (
    {
        "context": lambda x: get_context(x),
        "question": lambda x: x["question"]
    }
    | prompt
    | llm
    | StrOutputParser()
)

# Usage
query = "What are the impacts of climate change on biodiversity?"
answer = qa_chain.invoke({"question": query})

print(f"Question: {query}\n")
print("="*80)
print("ANSWER:")
print("="*80)
print(answer)
print("="*80)

Question: What are the impacts of climate change on biodiversity?

ANSWER:
According to the provided context, climate change has significant impacts on biodiversity in both terrestrial and marine ecosystems. The effects of climate change on biodiversity can be summarized as follows:

**Terrestrial Ecosystems:**

1. **Shifts in habitat ranges**: Climate change is altering the ranges of habitats, leading to changes in the distribution of plant and animal species.
2. **Changes in species composition**: Forests, grasslands, and deserts are experiencing shifts in the composition of plant and animal species, which can lead to a loss of biodiversity.
3. **Disruption of ecological balance**: These changes can disrupt the ecological balance, leading to a decline in ecosystem functions and services.

**Marine Ecosystems:**

1. **Rising sea temperatures**: Increased sea temperatures are affecting marine biodiversity, from coral reefs to deep-sea habitats.
2. **Ocean acidification**: The absorptio

In [25]:
# Cross-Encoder Retriever (also update for Pydantic v2)
class CrossEncoderRetriever(BaseRetriever, BaseModel):
    model_config = ConfigDict(arbitrary_types_allowed=True)  # Updated

    vectorstore: Any = Field(description="Vector store for initial retrieval")
    cross_encoder: Any = Field(description="Cross-encoder model for reranking")
    k: int = Field(default=5, description="Number of documents to retrieve initially")
    rerank_top_k: int = Field(default=3, description="Number of documents to return after reranking")

    def _get_relevant_documents(self, query: str) -> List[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=self.k)
        pairs = [[query, doc.page_content] for doc in initial_docs]
        scores = self.cross_encoder.predict(pairs)
        scored_docs = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)
        return [doc for doc, _ in scored_docs[:self.rerank_top_k]]

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError("Async retrieval not implemented")

### Create a custom retriever based on our reranker

In [37]:
from langchain_classic.chains import RetrievalQA
from langchain_core.retrievers import BaseRetriever
from pydantic import BaseModel, Field
from typing import Any, List
from langchain_core.documents import Document


class CustomRetriever(BaseRetriever, BaseModel):
    vectorstore: Any = Field(description="Vector store for initial retrieval")
    k: int = Field(default=4, description="Number of docs to return")

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(self, query: str) -> List[Document]:
        """Retrieve relevant docs from vectorstore."""
        return self.vectorstore.similarity_search(query, k=self.k)
    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError("Async retrieval not implemented")

    def get_relevant_documents(self, query: str, num_docs=2) -> List[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=30)
        return rerank_documents(query, initial_docs, top_n=num_docs)

# Create the custom retriever
custom_retriever = CustomRetriever(vectorstore=vectorstore)

# Create an LLM for answering questions
#llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
llm = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile",  max_tokens=4000)

# Create the RetrievalQA chain with the custom retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True
)


/tmp/ipython-input-1202176180.py:8: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class CustomRetriever(BaseRetriever, BaseModel):


### Example query


In [ ]:
result = qa_chain({"query": query})

print(f"\nQuestion: {query}")
print(f"Answer: {result['result']}")
print("\nRelevant source documents:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document

/tmp/ipython-input-546735038.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  result = qa_chain({"query": query})



Question: What are the impacts of climate change on biodiversity?
Answer: According to the context, climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species distributions, and impacting ecosystem functions, leading to a loss of biodiversity and disrupting ecological balance. In marine ecosystems, rising sea temperatures, ocean acidification, and changing currents are affecting marine biodiversity, from coral reefs to deep-sea habitats, and can disrupt marine food webs and fisheries. Additionally, species migration and changes in reproductive cycles can also disrupt ecosystems. Overall, climate change is causing a loss of biodiversity and disrupting the balance of ecosystems.

Relevant source documents:

Document 1:
Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species 
distributions, and impacting ecosystem functions. Forests, grasslands, and deserts are 
experiencing shi...

Document 2:
protection, and hab

### Example that demonstrates why we should use reranking

In [ ]:
chunks = [
    "The capital of France is great.",
    "The capital of France is huge.",
    "The capital of France is beautiful.",
    """Have you ever visited Paris? It is a beautiful city where you can eat delicious food and see the Eiffel Tower.
    I really enjoyed all the cities in france, but its capital with the Eiffel Tower is my favorite city.""",
    "I really enjoyed my trip to Paris, France. The city is beautiful and the food is delicious. I would love to visit again. Such a great capital city."
]
docs = [Document(page_content=sentence) for sentence in chunks]


def compare_rag_techniques(query: str, docs: List[Document] = docs) -> None:
    #embeddings = OpenAIEmbeddings()
    embeddings = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    vectorstore = FAISS.from_documents(docs, embeddings)

    print("Comparison of Retrieval Techniques")
    print("==================================")
    print(f"Query: {query}\n")

    print("Baseline Retrieval Result:")
    baseline_docs = vectorstore.similarity_search(query, k=2)
    for i, doc in enumerate(baseline_docs):
        print(f"\nDocument {i+1}:")
        print(doc.page_content)

    print("\nAdvanced Retrieval Result:")
    custom_retriever = CustomRetriever(vectorstore=vectorstore)
    advanced_docs = custom_retriever.get_relevant_documents(query)
    for i, doc in enumerate(advanced_docs):
        print(f"\nDocument {i+1}:")
        print(doc.page_content)


query = "what is the capital of france?"
compare_rag_techniques(query, docs)

Comparison of Retrieval Techniques
Query: what is the capital of france?

Baseline Retrieval Result:

Document 1:
The capital of France is huge.

Document 2:
The capital of France is great.

Advanced Retrieval Result:

Document 1:
I really enjoyed my trip to Paris, France. The city is beautiful and the food is delicious. I would love to visit again. Such a great capital city.

Document 2:
The capital of France is huge.


## Method 2: Cross Encoder models

<div style="text-align: center;">

<img src="../images/rerank_cross_encoder.svg" alt="rerank cross encoder" style="width:40%; height:auto;">
</div>

### Define the cross encoder class

In [ ]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

class CrossEncoderRetriever(BaseRetriever, BaseModel):
    vectorstore: Any = Field(description="Vector store for initial retrieval")
    cross_encoder: Any = Field(description="Cross-encoder model for reranking")
    k: int = Field(default=5, description="Number of documents to retrieve initially")
    rerank_top_k: int = Field(default=3, description="Number of documents to return after reranking")

    class Config:
        arbitrary_types_allowed = True

    def _get_relevant_documents(self, query: str) -> List[Document]:
        # Initial retrieval
        initial_docs = self.vectorstore.similarity_search(query, k=self.k)

        # Prepare pairs for cross-encoder
        pairs = [[query, doc.page_content] for doc in initial_docs]

        # Get cross-encoder scores
        scores = self.cross_encoder.predict(pairs)

        # Sort documents by score
        scored_docs = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)

        # Return top reranked documents
        return [doc for doc, _ in scored_docs[:self.rerank_top_k]]

    async def _aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError("Async retrieval not implemented")



/tmp/ipython-input-4207928631.py:3: PydanticDeprecatedSince20: Support for class-based `config` is deprecated, use ConfigDict instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.12/migration/
  class CrossEncoderRetriever(BaseRetriever, BaseModel):


### Create an instance and showcase over an example

In [26]:
# Create the cross-encoder retriever
cross_encoder_retriever = CrossEncoderRetriever(
    vectorstore=vectorstore,
    cross_encoder=cross_encoder,
    k=10,  # Retrieve 10 documents initially
    rerank_top_k=5  # Return top 5 after reranking
)

# Set up the LLM
#llm = ChatOpenAI(temperature=0, model_name="gpt-4o")
llm = ChatGroq(temperature=0, model_name="llama-3.3-70b-versatile",  max_tokens=4)

# Create the RetrievalQA chain with the cross-encoder retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=cross_encoder_retriever,
    return_source_documents=True
)

# Example query
query = "What are the impacts of climate change on biodiversity?"
result = qa_chain({"query": query})

print(f"\nQuestion: {query}")
print(f"Answer: {result['result']}")
print("\nRelevant source documents:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document


Question: What are the impacts of climate change on biodiversity?
Answer: According to the text

Relevant source documents:

Document 1:
Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species 
distributions, and impacting ecosystem functions. Forests, grasslands, and deserts are 
experiencing shi...

Document 2:
protection, and habitat creation. 
Climate-Resilient Conservation 
Conservation strategies must account for climate change impacts to be effective. This 
includes identifying climate refugia, areas le...

Document 3:
goals. Policies should promote synergies between biodiversity conservation and climate 
action. 
Chapter 10: Climate Change and Human Health 
Health Impacts 
Heat-Related Illnesses 
Rising temperature...

Document 4:
Local communities are often on the front lines of climate impacts and can be powerful agents 
of change. Community-based conservation projects involve residents in protecting and 
restoring natural re...

Docume

![](https://europe-west1-rag-techniques-views-tracker.cloudfunctions.net/rag-techniques-tracker?notebook=all-rag-techniques--reranking)